In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ann-car-sales-price-prediction/car_purchasing.csv


In [26]:
import pandas as pd

# Load dataset
df = pd.read_csv("/kaggle/input/ann-car-sales-price-prediction/car_purchasing.csv", encoding="ISO-8859-1")

# Display first few rows
print(df.head())
print(df.info())


     customer name                                    customer e-mail  \
0    Martina Avila  cubilia.Curae.Phasellus@quisaccumsanconvallis.edu   
1    Harlan Barnes                                eu.dolor@diam.co.uk   
2  Naomi Rodriquez  vulputate.mauris.sagittis@ametconsectetueradip...   
3  Jade Cunningham                            malesuada@dignissim.com   
4     Cedric Leach     felis.ullamcorper.viverra@egetmollislectus.net   

        country  gender        age  annual Salary  credit card debt  \
0      Bulgaria       0  41.851720    62812.09301      11609.380910   
1        Belize       0  40.870623    66646.89292       9572.957136   
2       Algeria       1  43.152897    53798.55112      11160.355060   
3  Cook Islands       1  58.271369    79370.03798      14426.164850   
4        Brazil       1  57.313749    59729.15130       5358.712177   

     net worth  car purchase amount  
0  238961.2505          35321.45877  
1  530973.9078          45115.52566  
2  638467.1773      

In [27]:
# Drop irrelevant columns
df.drop(columns=["customer name", "customer e-mail", "country"], inplace=True)

# Convert categorical variable (gender: 0 for Male, 1 for Female)
df["gender"] = df["gender"].astype(int)

# Define features (X) and target variable (y)
X = df.drop(columns=["car purchase amount"])
y = df["car purchase amount"]

# Feature Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert back to DataFrame
X = pd.DataFrame(X_scaled, columns=X.columns)

# Print dataset after preprocessing
print(X.head())
print(y.head())


     gender       age  annual Salary  credit card debt  net worth
0 -1.012073 -0.550749       0.058576          0.574271  -1.110469
1 -1.012073 -0.673834       0.386570         -0.009951   0.573929
2  0.988071 -0.387508      -0.712361          0.445452   1.193976
3  0.988071  1.509206       1.474794          1.382369   0.675595
4  0.988071  1.389066      -0.205111         -1.218962   0.743113
0    35321.45877
1    45115.52566
2    42925.70921
3    67422.36313
4    55915.46248
Name: car purchase amount, dtype: float64


In [28]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

# Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# Print R² Scores
print("Linear Regression R² Score:", r2_score(y_test, y_pred_lr))
print("Random Forest R² Score:", r2_score(y_test, y_pred_rf))


Linear Regression R² Score: 0.9999999806028682
Random Forest R² Score: 0.9498621195840186


In [29]:
def evaluate_model(y_test, y_pred, model_name):
    print(f"Model: {model_name}")
    print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
    print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
    print("Root Mean Squared Error:", np.sqrt(mean_squared_error(y_test, y_pred)))
    print("R² Score:", r2_score(y_test, y_pred))
    print("-" * 50)

evaluate_model(y_test, y_pred_lr, "Linear Regression")
evaluate_model(y_test, y_pred_rf, "Random Forest")


Model: Linear Regression
Mean Absolute Error: 1.1535708940223048
Mean Squared Error: 2.094369602944093
Root Mean Squared Error: 1.44719369917924
R² Score: 0.9999999806028682
--------------------------------------------------
Model: Random Forest
Mean Absolute Error: 1605.1012876119994
Mean Squared Error: 5413545.359877351
Root Mean Squared Error: 2326.702679733135
R² Score: 0.9498621195840186
--------------------------------------------------


In [38]:
import joblib
joblib.dump(rf, "sales_prediction_model.pkl")



['sales_prediction_model.pkl']

In [39]:
model = joblib.load("sales_prediction_model.pkl")
new_predictions = model.predict(X_test)
